RFE (Recursive Feature Elimination) and VIF (Variance Inflation Factor) are two different techniques used in feature selection and evaluation, but they serve distinct purposes and are used in different contexts.

# Recursive Feature Elimination

Purpose:
- RFE is a wrapper method for feature selection that aims to select the most important features by recursively eliminating the least important ones based on a model's performance (e.g., a linear regression, decision tree, or SVM).

How It Works:

- Start with all features in the dataset.
- Train a model (e.g., linear regression) and rank features based on their importance (e.g., coefficients in linear regression, feature importance in decision trees).
- Remove the least important feature(s).
- Repeat steps 2–3 until a specified number of features is selected.

Use Cases:
- To find the most predictive features for a model.
- Works with both linear and non-linear models.

Advantages:
- Considers the relationship between features and the target variable.
- Can handle non-linear models and complex feature interactions.

Limitations:
- Computationally expensive, especially for large datasets.
- Requires a model to evaluate feature importance, which introduces model dependency.


Let us prepare the data first where we can apply our feature selection process and compare our RFE and later VIF's performance on the same

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDRegressor
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, LinearSVC, SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import ConfusionMatrixDisplay, f1_score, make_scorer, confusion_matrix, mean_squared_error, mean_absolute_error

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from scipy.stats import randint, uniform
from sklearn.feature_selection import RFE

In [3]:
raw_data = pd.read_csv("/Users/macbookair/Downloads/car_insurance_claim.csv")

In [4]:
pd.set_option('display.max_columns', None)
raw_data.head()

,ID,KIDSDRIV,BIRTH,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,GENDER,EDUCATION,OCCUPATION,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,63581743,0,16MAR39,60.0,0,11.0,"$67,349",No,$0,z_No,M,PhD,Professional,14,Private,"$14,230",11,Minivan,yes,"$4,461",2,No,3,$0,18.0,0,Highly Urban/ Urban
1,132761049,0,21JAN56,43.0,0,11.0,"$91,449",No,"$257,252",z_No,M,z_High School,z_Blue Collar,22,Commercial,"$14,940",1,Minivan,yes,$0,0,No,0,$0,1.0,0,Highly Urban/ Urban
2,921317019,0,18NOV51,48.0,0,11.0,"$52,881",No,$0,z_No,M,Bachelors,Manager,26,Private,"$21,970",1,Van,yes,$0,0,No,2,$0,10.0,0,Highly Urban/ Urban
3,727598473,0,05MAR64,35.0,1,10.0,"$16,039",No,"$124,191",Yes,z_F,z_High School,Clerical,5,Private,"$4,010",4,z_SUV,no,"$38,690",2,No,3,$0,10.0,0,Highly Urban/ Urban
4,450221861,0,05JUN48,51.0,0,14.0,NaN,No,"$306,251",Yes,M,<High School,z_Blue Collar,32,Private,"$15,440",7,Minivan,yes,$0,0,No,0,$0,6.0,0,Highly Urban/ Urban


#   DATA CLEANING

In [5]:
data_df = raw_data.copy()

We have just created a copy of our actual data and changed the column names to little descriptive names.

In [6]:
col_names = {
    'KIDSDRIV': 'num_young_drivers',
    'BIRTH': 'date_of_birth',
    'AGE': 'age',
    'HOMEKIDS': 'num_of_children',
    'YOJ': 'years_job_held_for',
    'INCOME': 'income',
    'PARENT1': 'single_parent',
    'HOME_VAL': 'value_of_home',
    'MSTATUS': 'married',
    'GENDER': 'gender',
    'EDUCATION': 'highest_education',
    'OCCUPATION': 'occupation',
    'TRAVTIME': 'commute_dist',
    'CAR_USE': 'type_of_use',
    'BLUEBOOK': 'vehicle_value',
    'TIF': 'policy_tenure',
    'CAR_TYPE': 'vehicle_type',
    'RED_CAR': 'red_vehicle',
    'OLDCLAIM': '5_year_total_claims_value',
    'CLM_FREQ': '5_year_num_of_claims',
    'REVOKED': 'licence_revoked',
    'MVR_PTS': 'license_points',
    'CLM_AMT': 'new_claim_value',
    'CAR_AGE': 'vehicle_age',
    'CLAIM_FLAG': 'is_claim',
    'URBANICITY': 'address_type'
}

# Update column names
data_df.rename(columns=col_names, inplace=True)

In [7]:
# Drop duplicates
data_df.drop_duplicates(inplace=True)

In [8]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10301 entries, 0 to 10301
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         10301 non-null  int64  
 1   num_young_drivers          10301 non-null  int64  
 2   date_of_birth              10301 non-null  object 
 3   age                        10294 non-null  float64
 4   num_of_children            10301 non-null  int64  
 5   years_job_held_for         9753 non-null   float64
 6   income                     9731 non-null   object 
 7   single_parent              10301 non-null  object 
 8   value_of_home              9726 non-null   object 
 9   married                    10301 non-null  object 
 10  gender                     10301 non-null  object 
 11  highest_education          10301 non-null  object 
 12  occupation                 9636 non-null   object 
 13  commute_dist               10301 non-null  int64  


We have observed that some columns like income includes special characters like "$" and ",". We need to change that to change the column type to Integer

In [9]:
# Define currency based columns
currency_cols = ['income', 'value_of_home', 'vehicle_value', '5_year_total_claims_value', 'new_claim_value']

# Create function to remove '$' and ','
def format_currency_cols(data, cols):
    for col in cols:
        data[col] = data[col].replace('[\\$,]', '', regex=True).astype('Int64')
    return data

data_df = format_currency_cols(data_df, currency_cols)

In [10]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10301 entries, 0 to 10301
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         10301 non-null  int64  
 1   num_young_drivers          10301 non-null  int64  
 2   date_of_birth              10301 non-null  object 
 3   age                        10294 non-null  float64
 4   num_of_children            10301 non-null  int64  
 5   years_job_held_for         9753 non-null   float64
 6   income                     9731 non-null   Int64  
 7   single_parent              10301 non-null  object 
 8   value_of_home              9726 non-null   Int64  
 9   married                    10301 non-null  object 
 10  gender                     10301 non-null  object 
 11  highest_education          10301 non-null  object 
 12  occupation                 9636 non-null   object 
 13  commute_dist               10301 non-null  int64  


As we can see that all the data types have changed to integers as we removed the special characters.

In [11]:
# Define columns that have prefix
z_prefix_cols = ['married', 'gender', 'highest_education', 'occupation', 'vehicle_type', 'address_type']

# Create function to remove 'z_' prefix
def remove_prefix(data, cols):
    for col in cols:
        data[col] = data[col].replace('[z_]', '', regex=True)
    return data

data_df = remove_prefix(data_df, z_prefix_cols)

We need to remove this Z prefix data values from multiple columns as we can see in married, gender etc.

In [12]:
data_df.drop(['ID', 'date_of_birth'], axis=1, inplace=True)

- ID is just a unique identifier so is not needed
- data_of_birth duplicates the age feature (as age infers this information) so is not needed

In [13]:
data_df.head()

,num_young_drivers,age,num_of_children,years_job_held_for,income,single_parent,value_of_home,married,gender,highest_education,occupation,commute_dist,type_of_use,vehicle_value,policy_tenure,vehicle_type,red_vehicle,5_year_total_claims_value,5_year_num_of_claims,licence_revoked,license_points,new_claim_value,vehicle_age,is_claim,address_type
0,0,60.0,0,11.0,67349,No,0,No,M,PhD,Professional,14,Private,14230,11,Minivan,yes,4461,2,No,3,0,18.0,0,Highly Urban/ Urban
1,0,43.0,0,11.0,91449,No,257252,No,M,High School,Blue Collar,22,Commercial,14940,1,Minivan,yes,0,0,No,0,0,1.0,0,Highly Urban/ Urban
2,0,48.0,0,11.0,52881,No,0,No,M,Bachelors,Manager,26,Private,21970,1,Van,yes,0,0,No,2,0,10.0,0,Highly Urban/ Urban
3,0,35.0,1,10.0,16039,No,124191,Yes,F,High School,Clerical,5,Private,4010,4,SUV,no,38690,2,No,3,0,10.0,0,Highly Urban/ Urban
4,0,51.0,0,14.0,<NA>,No,306251,Yes,M,<High School,Blue Collar,32,Private,15440,7,Minivan,yes,0,0,No,0,0,6.0,0,Highly Urban/ Urban


In [14]:
# Define bins
bins = [0.0, 5000, 10_000, 15_000, 20_000, 25_000, 30_000, 35_000, 40_000, 45_000, 50_000, np.inf]
# Define bin labels
labels = np.arange(1, 12)

# Apply the bins using cut
data_df['claim_value_cat'] = pd.cut(data_df['new_claim_value'], bins = bins, labels= labels, include_lowest=True)

In [15]:
cols_to_drop = [
    'red_vehicle',
]

numerical_cols = ['num_young_drivers',
 'age',
 'num_of_children',
 'years_job_held_for',
 'income',
 'value_of_home',
 'commute_dist',
 'vehicle_value',
 'policy_tenure',
 '5_year_total_claims_value',
 '5_year_num_of_claims',
 'license_points',
 'vehicle_age']

# Define ordinal features
cat_cols_ord = ['highest_education']
# Define binary features
cat_cols_bin = ['single_parent', 'married', 'gender', 'type_of_use', 'licence_revoked', 'address_type']
# Define one-hot features
cat_cols_one_hot = ['occupation', 'vehicle_type']

# MODEL PIPELINE

In [16]:
# Custom transformer to drop specified columns
class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(columns=self.columns_to_drop)
    
    def get_feature_names_out(self, input_features=None):
        return None

In [17]:
class SqrtTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_transform):
        self.columns_to_transform = columns_to_transform
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.columns_to_transform] = np.sqrt(X[self.columns_to_transform])
        return X

    def get_feature_names_out(self, input_features=None):
        return input_features

In [18]:
from sklearn import set_config

# Set transformer output to df so we can reference columns names
set_config(transform_output='pandas')

In [19]:
# Define column dropper pipeline
cols_to_drop_pipeline = Pipeline([
    ('col_dropper', ColumnDropper(cols_to_drop))
])

skewed_features = ['income', 'value_of_home', 'commute_dist', 'vehicle_value', 'policy_tenure', 'license_points']

# Define numerical feature pipeline
num_pipeline = Pipeline([
    ('knn_imputer', KNNImputer(n_neighbors=2)),
    ('sqrt', SqrtTransformer(skewed_features)),
    ('scaler', StandardScaler()),
])

# Define rank of education levels
education_rank = [['<High School', 'High School', 'Bachelors', 'Masters', 'PhD']]

# Define ordinal categorical feature pipeline (highest_education feature)
cat_ord_pipeline = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('ord_encoder', OrdinalEncoder(categories=education_rank)),
])

# Define binary categorical feature pipeline
cat_bin_pipeline = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('binary_encoder', OrdinalEncoder()),
])

# Define one-hot categorical feature pipeline
cat_one_hot_pipeline = Pipeline([
    ('cat_simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first')),
])

# Define preprocessing pipeline with a column transformer
preprocess_pipeline = ColumnTransformer([
    ('drop_features', cols_to_drop_pipeline, cols_to_drop),
    ('num', num_pipeline, numerical_cols),
    ('cat_ord', cat_ord_pipeline, cat_cols_ord),
    ('cat_bin', cat_bin_pipeline, cat_cols_bin),
    ('cat_one_hot', cat_one_hot_pipeline, cat_cols_one_hot),
])

In [20]:
X = data_df.copy()
y = data_df['is_claim']

# Drop the target feature
X.drop(columns=['new_claim_value','is_claim'], inplace=True)

# Create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=X['claim_value_cat'])

In [21]:
# Use the same pipeline as the classifier model to preprocess X train
X_train_prepared = preprocess_pipeline.fit_transform(X_train)

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

name, clf =  ('XGBoost', XGBClassifier(random_state=0))

In [ ]:
clf.fit(X_train_prepared, y_train)
X_test_prepared = preprocess_pipeline.fit_transform(X_test)
y_pred = clf.predict(X_test_prepared)
perf_tuple = (name, f1_score(y_test, y_pred, average='weighted'))
print(perf_tuple)

('XGBoost', 0.7672991505941971)


Now, we check whether using Advance feature transformation techniques like RFE and VIF changes any accuracy scores of the model.

First, we will use RFE to decide which feature to eliminate and we will use decision tree classifier to reduce the run time 

In [22]:
rfe = RFE(
estimator=DecisionTreeClassifier(),
n_features_to_select=10
)

pipe = Pipeline([('rfe',rfe)])
transformed_df_rfe = pipe.fit_transform(X_train_prepared, y_train)
support = pipe.named_steps['rfe'].support_
drop_cols_rfe = list(X_train_prepared.columns[support])

In [ ]:
drop_cols_rfe

['num__age',
 'num__years_job_held_for',
 'num__income',
 'num__value_of_home',
 'num__commute_dist',
 'num__vehicle_value',
 'num__policy_tenure',
 'num__5_year_total_claims_value',
 'num__license_points',
 'num__vehicle_age']

In [ ]:
X_train_rfe = X_train_prepared.drop(['num__age',
 'num__years_job_held_for',
 'num__income',
 'num__value_of_home',
 'num__commute_dist',
 'num__vehicle_value',
 'num__policy_tenure',
 'num__5_year_total_claims_value',
 'num__license_points',
 'num__vehicle_age'], axis=1)

In [ ]:
X_test_rfe = X_test_prepared.drop(['num__age',
 'num__years_job_held_for',
 'num__income',
 'num__value_of_home',
 'num__commute_dist',
 'num__vehicle_value',
 'num__policy_tenure',
 'num__5_year_total_claims_value',
 'num__license_points',
 'num__vehicle_age'], axis=1)

In [ ]:
clf.fit(X_train_rfe, y_train)
# X_test_rfe = preprocess_pipeline.fit_transform(X_test_rfe)
y_pred = clf.predict(X_test_rfe)
perf_tuple = (name, f1_score(y_test, y_pred, average='weighted'))
print(perf_tuple)

('XGBoost', 0.7442920713355535)


The f1 score has come down from 76.73% to 74.43% that means there is negative impact on using RFE on this dataset

Now, we will try to apply VIF.

# Variance Inflation Factor

Purpose:
- VIF is a statistical measure used to detect multicollinearity among independent variables in regression models.

How It Works:

- For each feature, compute a regression where that feature is the dependent variable, and the rest are independent variables.
- Calculate the R-square value for the regression.
- Compute VIF using the formula:
    (VIF)i = 1/(1-Riˆ2)

Interpreting VIF:
- VIF = 1: No multicollinearity.
- VIF > 5: Moderate multicollinearity (thresholds vary, but common cutoff is 5 or 10).
- VIF > 10: High multicollinearity, problematic.


Use Cases:
- To identify and address multicollinearity in linear regression models.

Advantages:
- Easy to calculate.
- Helps improve regression model interpretability.

Limitations:
- Only applicable to linear models.
- Doesn't evaluate the relationship between features and the target variable, only among features.

In [24]:
def calc_vif(df):
    df_cols = df.columns
    vif_values = [
        variance_inflation_factor(df.values, i) for i in range(len(df_cols))
        ]
    return pd.DataFrame(zip(df_cols, vif_values),columns=['Variable','VIF'])

In [25]:
vif_df = calc_vif(X_train_prepared)
drop_cols_vif = vif_df.loc[vif_df['VIF']>5]['Variable'].values
transformed_df_rfe_vif = X_train_prepared.drop(drop_cols_vif,axis=1)
display(calc_vif(transformed_df_rfe_vif))

,Variable,VIF
0,num__num_young_drivers,1.315863
1,num__age,1.450899
2,num__num_of_children,2.027072
3,num__years_job_held_for,1.521501
4,num__income,2.742351
5,num__value_of_home,1.870488
6,num__commute_dist,1.010240
7,num__vehicle_value,1.806091
8,num__policy_tenure,1.005262
9,num__5_year_total_claims_value,1.675245


In [26]:
list(drop_cols_vif)

['cat_ord__highest_education', 'cat_bin__type_of_use', 'cat_bin__address_type']

In [42]:
X_train_VIF = X_train_prepared.drop(['cat_ord__highest_education', 'cat_bin__type_of_use', 'cat_bin__address_type'],axis =1)
X_test_VIF = X_test_prepared.drop(['cat_ord__highest_education', 'cat_bin__type_of_use', 'cat_bin__address_type'],axis =1)

In [43]:
clf.fit(X_train_VIF, y_train)
y_pred = clf.predict(X_test_VIF)
perf_tuple = (name, f1_score(y_test, y_pred, average='weighted'))
print(perf_tuple)

('XGBoost', 0.7373420130501036)


The f-1 Score is this case has also dropped significantly, This means we cannot use VIF also for this dataset.

# When to Use RFE vs. VIF


Use RFE when:

- Your goal is to select the best predictive features for a machine learning model.
- You are working with non-linear models or need model-dependent feature importance.

Use VIF when:

- You are building a linear regression model and want to detect and handle multicollinearity.
- You need a simple, statistical measure of feature redundancy.


## Both methods can complement each other. For example, you might first use VIF to remove multicollinear features and then apply RFE to select the most predictive ones.

It’s not uncommon for RFE (Recursive Feature Elimination) and VIF (Variance Inflation Factor) to suggest different columns to drop, as they are fundamentally different methods with distinct purposes and approaches. Here's why they might give different results:

1. Different Objectives:

- RFE: Focuses on the relationship between features and the target variable. It aims to identify the features that are most predictive of the target.
- VIF: Focuses on the relationship among the independent features. It identifies multicollinearity and highlights features that may distort model interpretability but not necessarily affect the target variable directly.

Reason for Different Results:
RFE might retain features with high multicollinearity (high VIF) because they have strong predictive power for the target, while VIF would suggest removing them to address multicollinearity.

2. Dependence on the Model in RFE:

- RFE is a model-dependent method. The importance of features is determined based on the specific algorithm used (e.g., linear regression, random forest, etc.). Different algorithms may rank feature importance differently, leading to a different set of features selected by RFE.

- Reason for Different Results:
The feature rankings in RFE depend on the model's biases and the algorithm's ability to handle multicollinearity. For instance:
    - Tree-based models (e.g., random forest) are less sensitive to multicollinearity and may select features that VIF suggests dropping.
    - Linear models in RFE are more likely to align with VIF results because both are sensitive to multicollinearity.

3. Multicollinearity's Role:

- RFE: Multicollinearity may not be a concern unless it directly affects the model's predictive performance. A highly correlated feature might still be retained if it contributes significantly to the target prediction.
- VIF: Treats multicollinearity as a red flag. Even if a feature is important for prediction, it will be flagged for removal if it is highly correlated with others.

Reason for Different Results:
A feature might be useful for prediction (retained by RFE) but redundant due to high correlation with another feature (flagged by VIF).

4. Thresholds and Criteria:

- RFE: Drops features based on feature importance rankings and the number of features specified for selection.
- VIF: Drops features based on a threshold for multicollinearity (e.g., VIF > 5 or 10).

Reason for Different Results:
* The criteria for dropping features are not directly comparable, as RFE focuses on target prediction and VIF on feature redundancy.

5. Scale of Data:

- RFE: Model performance might be affected if features are not scaled appropriately (e.g., for models like linear regression or SVM).
- VIF: Works purely on correlation, and scaling doesn't affect VIF values.

Reason for Different Results:
* Unscaled data might impact RFE results but leave VIF unaffected, leading to discrepancies.


# Which One to Trust?
-> If your goal is prediction: Focus on RFE, as it selects features based on their ability to predict the target variable.

-> If your goal is interpretation: Focus on VIF, as it ensures the model is interpretable by removing redundant features.


We now use another dataset and find out the effects of RFE and VIF individually and use to compare them to original method and evaluate their performance

In [48]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the Boston housing dataset
boston = fetch_california_housing(as_frame=True)
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ---- RFE ----
# Initialize Linear Regression model
lr = LinearRegression()

# RFE: Select top 5 features
selector = RFE(lr, n_features_to_select=5)
selector = selector.fit(X_train, y_train)

# Get selected features
rfe_selected_features = X_train.columns[selector.support_]
print("RFE Selected Features:")
print(rfe_selected_features)

# Train model with RFE-selected features
X_train_rfe = X_train[rfe_selected_features]
X_test_rfe = X_test[rfe_selected_features]
lr.fit(X_train_rfe, y_train)
y_pred_rfe = lr.predict(X_test_rfe)
rfe_rmse = np.sqrt(mean_squared_error(y_test, y_pred_rfe))
print(f"RFE Model RMSE: {rfe_rmse:.4f}")

# ---- VIF ----
# Calculate VIF for each feature
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

# Initial VIF calculation
vif_df = calculate_vif(X_train)
print("\nInitial VIF:")
print(vif_df)

# Remove features with high VIF (e.g., > 10)
while vif_df["VIF"].max() > 5:
    high_vif_feature = vif_df.loc[vif_df["VIF"].idxmax(), "feature"]
    print(f"Removing feature with high VIF: {high_vif_feature}")
    X_train = X_train.drop(columns=[high_vif_feature])
    X_test = X_test.drop(columns=[high_vif_feature])
    vif_df = calculate_vif(X_train)

print("\nFinal VIF:")
print(vif_df)

# Train model with VIF-selected features
lr.fit(X_train, y_train)
y_pred_vif = lr.predict(X_test)
vif_rmse = np.sqrt(mean_squared_error(y_test, y_pred_vif))
print(f"VIF Model RMSE: {vif_rmse:.4f}")

# ---- Compare RFE and VIF ----
print("\nComparison of RMSE:")
print(f"Original Model RMSE: {np.sqrt(mean_squared_error(y_test, LinearRegression().fit(X_train, y_train).predict(X_test))):.4f}")
print(f"RFE Model RMSE: {rfe_rmse:.4f}")
print(f"VIF Model RMSE: {vif_rmse:.4f}")


RFE Selected Features:
Index(['MedInc', 'AveRooms', 'AveBedrms', 'Latitude', 'Longitude'], dtype='object')
RFE Model RMSE: 0.7528

Initial VIF:
      feature         VIF
0      MedInc   11.831609
1    HouseAge    7.155405
2    AveRooms   46.792373
3   AveBedrms   48.332634
4  Population    2.915730
5    AveOccup    1.080609
6    Latitude  560.583263
7   Longitude  641.224254
Removing feature with high VIF: Longitude
Removing feature with high VIF: AveBedrms
Removing feature with high VIF: Latitude
Removing feature with high VIF: AveRooms

Final VIF:
      feature       VIF
0      MedInc  3.348454
1    HouseAge  2.890335
2  Population  2.068500
3    AveOccup  1.079622
VIF Model RMSE: 0.8122

Comparison of RMSE:
Original Model RMSE: 0.8122
RFE Model RMSE: 0.7528
VIF Model RMSE: 0.8122
